In [1]:
import os
import json
import uuid
import yaml
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables.history import RunnableWithMessageHistory


In [2]:
with open("./api_keys.json", 'r') as f:
    api_keys = json.loads(f.read())

for key in api_keys.keys():
    os.environ[key] = api_keys[key]

In [3]:
gemini = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-05-20",temperature=0.2)
testing_version = "concise"

In [4]:
def read_prompts(path_to_file:str, tested_version:str):
    with open(path_to_file, "r") as f:
        prompt_data = yaml.safe_load(f)
    prompt_list=[]
    for task in prompt_data["tasks"]:
        for v in task["versions"]:
            if v["version"] == tested_version:
                prompt_list.append(v["text"])
    return prompt_list

In [5]:
chat_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

chain = chat_prompt | gemini #pipe style, replaces LLMChain()

message_history = InMemoryChatMessageHistory()
chain_with_memory = RunnableWithMessageHistory(
    chain,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [6]:
prompts = read_prompts("./prompts_tests.yaml", testing_version)
print(prompts)
markdown_output = [f"# Tests of prompts for Conversational Code Generation - version {testing_version}\n"]

print("Executing prompts and collecting responses")
for i, p in enumerate(prompts, start=1):
    result = chain_with_memory.invoke({"input": p},config={"configurable": {"session_id": str(uuid.uuid4())}})
    markdown_output.append(f"## Prompt {i}\n")
    markdown_output.append(f"**User:** {p}\n")
    markdown_output.append(f"**LLM Response:**\n\n{result.content.strip()}\n")
print("Prompt execution finished")


file_path = f"./test_results/test_conversation_version_{testing_version}.md"
with open(file_path, "w", encoding="utf-8") as f:
    f.write("\n".join(markdown_output))
print("Result saved to file")

['Implement bubble sort in Python.', 'Read numeric values from a CSV file in Python.', 'Read numbers from CSV and sort them using bubble sort.']
Executing prompts and collecting responses
Prompt execution finished
Result saved to file
